# Convert ROOT files to HDF5

The ROOT-digi files contain the calibrated events in NeuLAND. Here we extract the three basic properties number of hits, number of clusters, and total deposited energy and save them together with the number of incoming neutrons, number of primary points, and number of primary hits to a HDF5 file for basic reconstruction.

In [1]:
import ROOT
import h5py
import numpy as np

Welcome to JupyROOT 6.16/00


To create the HDF5 file, it is good to know the number of events to store ahead of time. With ROOT, this requires a bit more code than it should, as you can't have multiple files opened.

In [2]:
def get_num_events(infiles):
    # List Comprehension does not work with ROOT
    num_events = 0
    for _, filename in infiles:
        tfile = ROOT.TFile.Open(filename)
        ttree = tfile.Get("evt")
        num_events += ttree.GetEntries()
    return num_events

In [3]:
def create_hdf5(infiles, outfile):
    dim = 6
    num_events = get_num_events(infiles)
    chunksize = 1000

    print(f"->     Writing to {outfile}")
    with h5py.File(outfile, "w") as h5file:
        data = h5file.create_dataset(
            "data",
            shape=(num_events, dim),
            dtype=np.int16,
            chunks=(chunksize, dim),
            compression="gzip",
            compression_opts=9,
        )

        buff = np.zeros((dim), np.int16)
        i = 0
        for nIn, filename in infiles:
            print(f"Reading ROOT file {filename}")
            tfile = ROOT.TFile.Open(filename)
            ttree = tfile.Get("evt")
            for event in ttree:

                # nPN: Number of incoming primary neutrons
                buff[0] = nIn
                # nPP: Number of primary neutrons with an energy deposition in NeuLAND
                buff[1] = event.NeulandPrimaryPoints.GetEntries()
                # nPH: Number of hits that correspond to a energy deposition of a primary neutron
                buff[2] = event.NeulandPrimaryHits.GetEntries()
                # nHits: Number of hits
                buff[3] = event.NeulandHits.GetEntries()
                # nClus: Number of clusters
                buff[4] = event.NeulandClusters.GetEntries()
                # Edep: Total deposited (detected) energy
                buff[5] = round(sum([hit.GetE() for hit in event.NeulandHits]))

                data[i] = buff
                i += 1

In [4]:
incoming_neutrons = range(1, 6)
beam_energy = 600
for num_dp in [15, 30]:
    for dataset in ["training", "validation", "test"]:
        infiles = [
            (
                neutrons,
                f"simulation/{dataset}_{beam_energy}AMeV_{num_dp}dp_{neutrons}n.digi.root",
            )
            for neutrons in incoming_neutrons
        ]
        outfile = f"data/{dataset}_{beam_energy}AMeV_{num_dp}dp.h5"
        create_hdf5(infiles, outfile)